# Model

Folder Structure:

#### Root
- Data/
- - Test/                      # Contains evaluation images
- - Train/                     # Contains training images
- - sample_submission.csv      # Sample submission file for Kaggle competition
- - train_labels.csv           # csv file containing training set labels in format (fileName, label)
- Model.ipynb                  # Current file


In [64]:
from __future__ import print_function, division

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets

class PCamDataset(Dataset):
    """Histopathologic Cancer Detection Dataset"""
    def __init__(self, csv_file, root_dir, transform):
        
        """
        Args:
            csv_file (string): Path to the csv file with train labels.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train_labels = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.train_labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.train_labels.iloc[idx, 0])
        img_name = img_name + '.tif'
        image = Image.open(img_name)
        label = self.train_labels.iloc[idx, 1:].values
        return self.transform(image), int(label)

class Net(nn.Module):
    """Convolutional Neural Network"""
    def __init__(self):
        super(Net, self).__init__()
        # 1. Convolutional layers
        self.conv1 = nn.Conv2d( in_channels = 3
                               , out_channels = 16
                               , kernel_size = 3
                               , stride = 1
                               , padding = 1 )
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d( in_channels = 16
                               , out_channels = 32
                               , kernel_size = 3
                               , stride = 1
                               , padding = 1 )
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d( in_channels = 32
                               , out_channels = 64
                               , kernel_size = 3
                               , stride = 1
                               , padding = 1 )
        self.bn3 = nn.BatchNorm2d(64)
        self.pool = nn.MaxPool2d( kernel_size = 2
                                 , stride = 2
                                 , padding = 0 )

        # 2. FC layers to final output
        self.fc1 = nn.Linear(in_features = 64*12*12, out_features = 512)
        self.fc_bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(in_features = 512, out_features = 256)
        self.fc_bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(in_features = 256, out_features = 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Apply 3x...
        # Convolution Layers, followed by Batch Normalizations, Maxpool, and ReLU
        x = self.bn1(self.conv1(x))                      # batch_size x 96 x 96 x 16
        x = self.pool(F.relu(x))                         # batch_size x 48 x 48 x 16
        x = self.bn2(self.conv2(x))                      # batch_size x 48 x 48 x 32
        x = self.pool(F.relu(x))                         # batch_size x 24 x 24 x 32
        x = self.bn3(self.conv3(x))                      # batch_size x 24 x 24 x 64
        x = self.pool(F.relu(x))                         # batch_size x 12 x 12 x 64
        
        # Flatten the output for each image
        x = x.view(-1, self.num_flat_features(x))        # batch_size x 12*12*64
        
        # Apply 3 FC Layers
        x = self.fc1(x)
        x = self.fc_bn1(x)
        x = F.relu(x)
        x = self.fc_bn2(self.fc2(x))
        x = F.relu(x)
        x = self.fc3(x)
        return self.sigmoid(x)

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
def show_image(image, label):
    """Show image with label"""
    print('Label: ' + str(label[0]))
    plt.imshow(image)

In [65]:
print(Net())


Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=9216, out_features=512, bias=True)
  (fc_bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc_bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=256, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [76]:
# Set parameters
train_label_file = 'Data\\train_labels.csv'
train_set_dir = 'Data\\Train'
batch_size = 2
num_workers = 0
dev_split = 0.1                              # 90/10 Train/Dev split
random_seed = 0                              # Set a random seed so the shuffle is predictable each run
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.003, momentum = 0.9)
train_on_gpu = False
train_loss_min = np.Inf
early_stop_limit = 20
bad_epoch_count = 0
num_epochs = 10
stop = False

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create DataSet
dataset = PCamDataset( csv_file = train_label_file
                      , root_dir = train_set_dir
                      , transform = transform )

# Create data indices for train/dev set split
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(dev_split * dataset_size))
np.random.seed(random_seed)
np.random.shuffle(indices)
train_indices, dev_indices = indices[split:], indices[:split]

######################################################
### Starting with smaller train_sampler to ramp up ###
######################################################
temp_train_split = 10
train_indices = indices[:temp_train_split]
dev_indices = indices[temp_train_split:temp_train_split+10]
######################################################

# Create data samplers and loaders
train_sampler = SubsetRandomSampler(train_indices)
dev_sampler = SubsetRandomSampler(dev_indices)

train_loader = DataLoader( dataset = dataset
                          , batch_size = batch_size
                          , num_workers = num_workers
                          , sampler = train_sampler )
dev_loader = DataLoader( dataset = dataset
                          , batch_size = batch_size
                          , num_workers = num_workers
                          , sampler = dev_sampler )

# Train
model = Net()
if (train_on_gpu):
    model.cuda()
    
# Loop over the dataset multiple times
for epoch in range(num_epochs):
    
    # Keep track of training loss
    train_loss = 0.0

    # Train the model
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        if train_on_gpu:
            data, target = data.cuda(), label.cuda()
        else:
            data, target = image, label
            
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Calculate the batch's loss
        loss = criterion(output, target.float())
        
        # Backward pass
        loss.backward()
        
        # Perform a single optimization step to update parameters
        optimizer.step()
        
        # Update the training loss
        train_loss += loss.item()
                  
    # Calculate average loss
    train_loss = train_loss/len(train_loader.dataset)
                  
    # Save model if train loss has decreased
    if train_loss <= train_loss_min:
        print('Dev loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        train_loss_min,
        train_loss))
        torch.save(model.state_dict(), 'model.pt')
        train_loss_min = train_loss
        bad_epoch_count = 0
    # If train loss didn't improve, increase bad_epoch_count and stop if bad_epoch_count >= early_stop_limit (early stop)
    else:
        bad_epoch_count += 1
        print('{} epochs of increasing dev loss'.format(bad_epoch_count))
        if (bad_epoch_count >= early_stop_limit):
            print('Stopping training')
            stop = True
            break
        
    if (stop):
        break

G:\Anaconda3\envs\pytorch_env\lib\site-packages\torch\nn\functional.py:2016: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Dev loss decreased (inf --> 0.000016).  Saving model ...
Dev loss decreased (0.000016 --> 0.000015).  Saving model ...
1 epochs of increasing dev loss
Dev loss decreased (0.000015 --> 0.000014).  Saving model ...
1 epochs of increasing dev loss
2 epochs of increasing dev loss


KeyboardInterrupt: 

In [57]:
from torch.autograd import Variable
a = torch.empty(1, 1, dtype=torch.float)
a.fill_(0.5152)
print(a)

t = torch.empty(1, 1, dtype=torch.float)
t.fill_(0)
print(t)
print(nn.BCELoss()(a,t)) # Different numbers

tensor([[0.5152]])
tensor([[0.]])
tensor(0.7240)
